# Standard Atmospheres

A number of 'standard' atmospheres were defined to cover the expected range of climatic conditions to be used in design analyses and optimisations.  This document defines and analyses the Modtran-calculated transmittance data.  Modtran 5.0 was used in these calculations.

## Introduction

The ground height for all atmospheric models is at sea level. 

The paths were calculated from the stated altitude at a slant angle of 45 deg down to earth (zenith angle of 135 degrees), but then normalised to a 1 km path length (irrespective of the actual path length).  This normalised path length is then later used to calculate the transmittance along the actual path length.

Within the constraints of a few assumptions, the transmittance for a path at a slant angle of -45 degrees can be used to calculate the transmittance at any other downward slant path angle.  These constraints are   
 1. The new path starts at the same altitude for which the original data was calculated.      
 2. The earth is flat, and hence the curvature of atmospheric profiles are ignored.    
 3. The path length is adapted to account for the different path length at a different slant angle.  
 4. The path intersects the ground, even if at infinity for slant angles minutely below the horizontal.


## Data formats

The path definitions and data are stored in the TAPM Stash repository at  
`TBC`

The following data files are available:

__./scenario/alt/scenario-altm.1km__   
Atmo transmittance normalised to 1 km path length, irrespective of actual path length.
Space delimited text file, col0=wavelength in um, col1=transmittance
First line describes the file scenario and altitude. 
These files conform to the NV-IPM vector file format.

__./scenario/scenario.png__  
A plot of normalised transmittance values for all altitudes.

__./AllScen-altm.png__  
A plot of normalised transmittance values for all scenarios at the stated altitude.

__./scenario/scenario.xlsx__  
An Excel spreadsheet of normalised transmittance values for all altitudes for stated scenario.

## Atmosphere Scenarios

### ExtremeHotLowHumidity
__very hot, low humidity, 75 km Desert visibility - no wind, clear day in the desert__  
User model (36 layers), 70 km visibility  Desert aerosol, ground level P=1030~mbar, T=44C, RH=30%, 18.1~g/m$^3$  absolute humidity. 

### ExtremeHumidity:
__very high humidity - Arabian gulf, tropical Africa / Amazon__  
User model (36 layers), 23 km visibility  Rural aerosol, ground level P=1030~mbar, T=35C, RH=95%, 37.9~g/m$^3$ absolute humidity. 
At sea level, this atmosphere corresponds with  the highest recorded dew point at 34~$^\circ$C.  MIL-HDBK-310 indicates that the probability of absolute humidity exceeding this level is significantly less than 1\%. This corresponds to the highest recorded humidity ever, recorded in Sharjah in the UAE.

### MidLatMaritimeSummer:
__Mediterranean coastal summer day good visibility__  
Mid-Latitude Summer, 23 km visibility  Maritime aerosol, ground level P=1012~mbar, T=21C, RH=76%, 14~g/m$^3$ absolute humidity. 

### MidLatMaritimeWinter:
__Mediterranean coastal winter day poor visibility__  
Mid-Latitude Winter, 10 km visibility  Maritime aerosol, ground level P=1018~mbar, T=-1C, RH=77%, 3~g/m$^3$ absolute humidity. 

### ScandinavianSummer:
__Scandinavian summer day, windy conditions__  
Sub-Arctic Summer, 31 km visibility  Navy Maritime aerosol (wind speed 20 m/s, 24 hr ave 8 m/s), ground level P=1010~mbar, T=14C, RH=75%, 9~g/m$^3$ absolute humidity. 

### ScandinavianWinter:
__Scandinavian winter day, windy conditions__  
Sub-Arctic Winter, 31 km visibility  Navy Maritime aerosol (wind speed 20 m/s, 24 hr ave 8 m/s), ground level P=1010~mbar, T=-15.9C, RH=80%, 1~g/m$^3$ absolute humidity. 

### TropicalDesert:
__Moderate temperature, 75 km Desert visibility - no wind, clear day in the desert__  
Tropical, 75 km visibility  Desert aerosol, ground level P=1013~mbar, T=26.5C, RH=75%, 18.8~g/m$^3$ absolute humidity. 

### TropicalRural:
__Moderate temperature, 23 km Rural visibility__  
Tropical, 23 km visibility  Rural aerosol, ground level P=1013~mbar, T=26.5C, RH=75%, 18.8~g/m$^3$ absolute humidity. 

### TropicalUrban:
__Moderate temperature, 5 km Urban visibility__  
Tropical, 5 km visibility  Urban aerosol, ground level P=1013~mbar, T=26.5C, RH=75%, 18.8~g/m$^3$ absolute humidity. 

### USStdNavyMarVis23km:
__Moderately Low temperature, moderate humudity, 23 km Navy Maritime visibility__  
US Standard, 23 km visibility  Navy Maritime aerosol, ground level P=1013~mbar, T=15C, RH=46%, 5.9~g/m$^3$ absolute humidity.  Windspeed of 7.4 m/s corresponds to a Beaufort sea state 4.

# Preparing for Analysis

In [ ]:
#to prepare the environment
import numpy as np
import scipy as sp
import pandas as pd
import pyradi.ryplot as ryplot
import os.path
from scipy.optimize import curve_fit

%matplotlib inline

# %reload_ext autoreload
# %autoreload 2

import pyradi.ryplot as ryplot
import pyradi.ryplanck as ryplanck
import pyradi.ryfiles as ryfiles
import pyradi.rymodtran as rymodtran

# import xlsxwriter

from IPython.display import HTML
from IPython.display import Image
from IPython.display import display
from IPython.display import FileLink, FileLinks

#make pngs at 150 dpi
import matplotlib as mpl
mpl.rc("savefig", dpi=150)
mpl.rc('figure', figsize=(10,8))
# %config InlineBackend.figure_format = 'svg'

pd.set_option('display.max_columns', 80)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', 150)



Load the standard atmospheric atmospheres definitions.

In [ ]:
#to define the atmospheres
"""This file provides Python-style definitions of the standard atmospheres.
This file can be included in other python scripts
"""

#each base tape5 file is in its own directory
dirs = ['ExtremeHotLowHumidity', 'ExtremeHumidity',
        'MidLatMaritimeSummer','MidLatMaritimeWinter',
        'ScandinavianSummer','ScandinavianWinter',
        'TropicalDesert','TropicalRural',
        'TropicalUrban','USStdNavyMarVis23km']
        
#altitudes  (in m)
alts = [305, 1524, 3048, 4572, 6096, 7620, 9144, 10668, 12192, 13716, 14326, 15240]
              
atmospheres = {
    u'ExtremeHotLowHumidity':['44 C, 30% RH (18.1), 77 km Vis Desert'],
    u'ExtremeHumidity'      :['35 C, 95% RH (37.9), 23 km Vis Rural'],
    u'MidLatMaritimeSummer' :['21 C, 76% RH (14), 23 km Vis Maritime'],
    u'MidLatMaritimeWinter' :['-1 C, 77% RH (3), 10 km Vis Maritime'],
    u'ScandinavianSummer'   :['14 C, 75% RH (9), 31 km Vis Maritime'],
    u'ScandinavianWinter'   :['-15.9 C, 80% RH (1), 31 km Vis Maritime'],
    u'TropicalDesert'       :['26.6 C, 75% RH (18), 75 km Vis Desert'], 
    u'TropicalRural'        :['26.6 C, 75% RH (18), 23 km Vis Rural'],
    u'TropicalUrban'        :['26.6 C, 75% RH (18), 5 km Vis Urban'],
    u'USStdNavyMarVis23km'  :['15 C, 46% RH (5.9), 23 km Vis Nav.Mar, SS4'],
    }


# Atmospheric Vertical Profiles

The vertical profiles are extracted from the tape6 files.

In [ ]:
#to plot the vertical profiles
def plotProfiles(alts, dirs, atmospheres,maxalt):
    #first plot the different altitudes for each atmosphere
    pa = ryplot.Plotter(4, 1, 1, figsize=(6,6));
    pp = ryplot.Plotter(1, 1, 1, figsize=(6,6));
    pt = ryplot.Plotter(2, 1, 1, figsize=(6,6));
    ph = ryplot.Plotter(3, 1, 1, figsize=(6,6));
    
    for i,directory in enumerate(dirs):
        for alt in alts:
            if alt == 305:
                datadir = os.path.join('.',directory,'{}'.format(alt))
                filename = os.path.join(datadir,'tape6')
                with open(filename,'rt') as fin:
                    lines = fin.readlines()
                    cntProfiles = 0
                    linecnt = 0
                    doLines = True
                    while(doLines and linecnt < len(lines)):
                        if 'ATMOSPHERIC PROFILES' in lines[linecnt]:
                            linecnt += 2 # discard empty line and read next
                            acclines = []
                            if 'RH (%)' in lines[linecnt]:
                                linecnt += 2 # discard header lines
                                for i in range(36):
                                    acclines.append((lines[linecnt]).split())
                                    linecnt += 1
                                arr = np.asarray(acclines, dtype=np.float)
                                arr[0,1] = .1
                                #write profile to disk
                                profile = np.hstack((arr[:,1].reshape(-1,1),arr[:,2].reshape(-1,1),arr[:,3].reshape(-1,1),arr[:,10].reshape(-1,1)))
                                profilepath = os.path.join('.',directory,'{}-profile.txt'.format(directory))
                                np.savetxt(profilepath, profile)
                                #process
                                arrdf = pd.DataFrame(arr)
                                arrdf = arrdf[arrdf[1] <= maxalt]
                                arr = arrdf.values
                                aero = np.sum(arr[:,4:8], axis=1)
                                pa.logLog(1,aero,arr[:,1],label=[directory])
                                pp.semilogY(1,arr[:,2],arr[:,1],label=[directory])
                                pt.semilogY(1,arr[:,3],arr[:,1],label=[directory])
                                ph.semilogY(1,arr[:,10],arr[:,1],label=[directory])
                                doLines = False
                        linecnt += 1
        
    cpa = pa.getSubPlot(1)
    cpp = pp.getSubPlot(1)
    cpt = pt.getSubPlot(1)
    cph = ph.getSubPlot(1)
    
    for falt in [1000, 5000, 30000]:
        malt = 0.3048 * falt / 1000.
        pa.logLog(1,np.asarray([1e-4,1e0]),np.asarray([malt,malt]),plotCol='k')
        cpa.text(0.2,0.9*malt, '{:.0f} kft'.format(falt/1000.), horizontalalignment='left', fontsize=8)    
        pp.semilogY(1,np.asarray([240,1040]),np.asarray([malt,malt]),plotCol='k')
        cpp.text(960,0.9*malt, '{:.0f} kft'.format(falt/1000.), horizontalalignment='left', fontsize=8)    
        pt.semilogY(1,np.asarray([210,330]),np.asarray([malt,malt]),plotCol='k')
        cpt.text(318,0.9*malt, '{:.0f} kft'.format(falt/1000.), horizontalalignment='left', fontsize=8)    
        ph.semilogY(1,np.asarray([10,100]),np.asarray([malt,malt]),plotCol='k')
        cph.text(92,0.9*malt, '{:.0f} kft'.format(falt/1000.), horizontalalignment='left', fontsize=8)    

    for ext in np.logspace(-2,0, 20):#[1e-2, 1e-1, 1]:
        cpa.text(ext,0.105, '{:.0f} km vis'.format(3.91/ext), verticalalignment='bottom', horizontalalignment='center', fontsize=8, rotation=90)        
        
    cpa.set_xlabel('Extinction [km$^{-1}$]')
    cpa.set_ylabel('Altitude [km]')
    cpa.set_title('Vertical 550 nm Extinction Profiles')
    cpp.set_xlabel('Pressure [mBar]')
    cpp.set_ylabel('Altitude [km]')
    cpp.set_title('Vertical Pressure Profiles')
    cpt.set_xlabel('Temperature [K]')
    cpt.set_ylabel('Altitude [km]')
    cpt.set_title('Vertical Temperature Profiles')
    cph.set_xlabel('Relative humidity [%]')
    cph.set_ylabel('Altitude [km]')
    cph.set_title('Vertical Humidity Profiles')
#     pa.saveFig('profiles-aerosol.png')
#     pp.saveFig('profiles-pressure.png')
#     pt.saveFig('profiles-temperature.png')
#     ph.saveFig('profiles-humidity.png')
    
plotProfiles(alts, dirs, atmospheres, 10.)    

# Spectral Transmittance Plots

Plot the spectral transmittance for 1-km path length. Two sets of plots are shown:   
 1. For each atmosphere, plot the transmittance at different altitudes.   
 2. For shared altitudes, plot the different atmosphere's transmittance.  

In [ ]:
#to plot the spectral transmittance
def plotTau(alts, dirs, atmospheres):
  #first plot the different altitudes for each atmosphere
  for i,directory in enumerate(dirs):
    p = ryplot.Plotter(i, 1, 1, figsize=(12,6))
    for alt in alts:
      datadir = os.path.join('.',directory,'{}'.format(alt))
      filename = '{}-{}m.1km'.format(directory, alt)
      data = np.loadtxt(os.path.join(datadir,filename), skiprows=1)
      p.plot(1, data[:,0], data[:,1],
        '{} {} 1 km transmittance, 135 deg zenith'.format(directory, atmospheres[directory][0]),
        'Wavelength $\mu$m','Transmittance', label=['{} m'.format(alt)],legendAlpha=0.5)
#     p.saveFig(os.path.join('.',directory,'{}.png'.format(directory)))
    
  #now plot the different atmospheres for each altitude
  for j, alt in enumerate(alts):
    p = ryplot.Plotter(j+len(dirs), 1, 1, figsize=(12,6))
    for i,directory in enumerate(dirs):
      datadir = os.path.join('.',directory,'{}'.format(alt))
      filename = '{}-{}m.1km'.format(directory, alt)
      data = np.loadtxt(os.path.join(datadir,filename), skiprows=1)
      p.plot(1, data[:,0], data[:,1],
        '{} m altitude, 135 deg zenith 1 km transmittance'.format(alt),
        'Wavelength $\mu$m','Transmittance', label=['{} {}'.format(directory, atmospheres[directory][0])],
        legendAlpha=0.5)
#     p.saveFig(os.path.join('.','AllScen-{}m.png'.format(alt)))

plotTau(alts, dirs, atmospheres)

# Apparent Atmospheric Temperature

The apparent temperature of the atmosphere is a complex function of the temperature profile along the path.  For slant paths in inhomogeneous atmospheres the effective temperature is affected stronger by the path sections closer to the sensor than path sections further away.  

It can be shown that the path radiance in the infrared can be approximated by

\begin{equation}
L_\textrm{path} = (1-\tau)\,L(T_\textrm{bb})
\end{equation}

This implies that if $L_\textrm{path}$ and $\tau$ are known $T_\textrm{bb}$ can be determined.  The solution is determined by fitting a curve of the above shape to the Modtran-calculated path radiance, finding $T_\textrm{bb}$ for the best fit.  Only the thermal path radiance is used, which has most of its optical flux at wavenumbers below 2500 cm$^{-1}$.

The analysis below considers three spectral bands: (1) a band in the 3.5-4.8 $\mu$m range, (1) a band in the 8.3-12 $\mu$m range, and (3) a band that covers _four_ windows in the 3.2-12.5 $\mu$m range.  

It turns out that the temperatures calculated for the 3.5-4.8 $\mu$m band is similar to the average or broadband temperature, because of the mix of CO$_2$ absorption and good transmittance in this band.

At low altitudes all the atmospheric temperatures are similar but at high altitudes there can be a significant spread in the values.

The term 'best fit' is not easily defined.  It is not clear what a simple fit through all the data really mean.  Several options were considered: (1) use only the spectral ranges with high transmittance, (2) use only the spectral ranges with poor transmittance, (3) use all the data, and (4) calculate the mean value of the first two options.
The results of these two approaches are shown below in the example graphs.  The temperatures calculated from all the data and the mean of the high and low transmittance temperatures are broadly in agreement. 

If the apparent atmosphere temperatures are used in applications where the path radiance in the atmospheric windows are required, it is proposed that the 'high transmittance' temperature values be used. These values will be more applicable to the in-band path radiance.


The results shown in the graphs and tables below are for the following conditions:  

|Heading|Meaning|
|--|--|
|TmpLoTau|Apparent temperature in _all_ spectral ranges with low transmittance|
|TmpHiTau|Apparent temperature in _all_ spectral ranges with high transmittance|
|3.5-4.8|Apparent temperature in the full spectral band (including absorption)|
|8.3-12|Apparent temperature in the full spectral band (including absorption)|
|TmpAve|Arithmetic Average (TmpLoTau+TmpHiTau)/2|
|TmpAll|Apparent temperature over the total spectral range|


In [ ]:
#to return Placnk radiator radiance
def lpathfunc(nu,tbb):
    return  ryplanck.planck(nu,tbb, 'en') / np.pi 

In [ ]:
#to calculate and plot atmospheric temperature
files = ryfiles.listFiles(root=r'.', 
                patterns=r'.*tape7', recurse=1, return_folders=0, useRegex=True)

#remove the files with elev in the name
fileslist = []
for filen in files:
    if 'elev' not in filen:
        fileslist.append(filen)


dfAtmoTempCols = ['Atmosphere','Altitude','TmpLoTau','TmpHiTau','3.5-4.8','8.3-12','TmpAve','TmpAll']
dfAtmoTemp = pd.DataFrame(columns=dfAtmoTempCols)

# for i,filename in enumerate([r'C:\WorkA\TAPM\Atmospheres\Standard\ExtremeHotLowHumidity\305\tape7',
#                             r'C:\WorkA\TAPM\Atmospheres\Standard\ExtremeHotLowHumidity\15240\tape7']):
for i,filename in enumerate(fileslist):
    filesplit = filename.split('\\')
    colselect = ['FREQ', 'TOT_TRANS','PTH_THRML']
    tape7 = rymodtran.loadtape7(filename, colselect )
    tape7[:,2] *= 1e4

    genericbands = {
        '10-12,5':[12.5, 10.],  # generic
        '8.33-9.34':[9.34, 8.33],  # generic
        '4.5-5.0':[5.0, 4.5],  # generic
        '3.2-4.5':[4.17, 3.2],  # generic
        }
    camerabands = {    
        '8.33-12.0':[12.0, 8.33], # this is the 8-12 band including 
        '3.5-4.8':[4.8, 3.5], # this is the 3-5 band including CO2
        }
    
    #prepare selection for generic bands
    selectHiTau = np.zeros(tape7[:,0].shape)
    for key in genericbands:
        selectHiTau = np.logical_or(selectHiTau, 
            np.all([ tape7[:,0] >= 1e4/genericbands[key][0], tape7[:,0] <= 1e4/genericbands[key][1]], axis=0))

    selectLoTau = np.logical_not(selectHiTau)
    select3to5 = np.all([ tape7[:,0] >= 1e4/camerabands['3.5-4.8'][0], tape7[:,0] <= 1e4/camerabands['3.5-4.8'][1]], axis=0)
    select8to12 = np.all([ tape7[:,0] >= 1e4/camerabands['8.33-12.0'][0], tape7[:,0] <= 1e4/camerabands['8.33-12.0'][1]], axis=0)
   
    popt, pcov = curve_fit(lpathfunc,tape7[selectHiTau,0], tape7[selectHiTau,2]/(1-tape7[selectHiTau,1]), p0=(250.) )
    tmpHiTau = popt[0]
    popt, pcov = curve_fit(lpathfunc,tape7[selectLoTau,0], tape7[selectLoTau,2]/(1-tape7[selectLoTau,1]), p0=(250.) )
    tmpLoTau = popt[0]
    tmpAve = (tmpHiTau + tmpLoTau) / 2.
    popt, pcov = curve_fit(lpathfunc,tape7[:,0], tape7[:,2]/(1-tape7[:,1]), p0=(250.) )
    tmpAll = popt[0]
    popt, pcov = curve_fit(lpathfunc,tape7[select3to5,0], tape7[select3to5,2]/(1-tape7[select3to5,1]), p0=(250.) )
    tmp3to5 = popt[0]
    popt, pcov = curve_fit(lpathfunc,tape7[select8to12,0], tape7[select8to12,2]/(1-tape7[select8to12,1]), p0=(250.) )
    tmp8to12 = popt[0]
    
    dfAtmoTemp = dfAtmoTemp.append(pd.DataFrame([[filesplit[-3], float(filesplit[-2]), tmpLoTau, tmpHiTau, 
                            tmp3to5, tmp8to12, tmpAve, tmpAll]], columns=dfAtmoTempCols))

    if filesplit[-3] =='ExtremeHotLowHumidity' and (filesplit[-2]=='305' or filesplit[-2]=='15240') :

        p = ryplot.Plotter(i, 2,2,'{} - {} m'.format(filesplit[-3],filesplit[-2]),figsize=(10,10))
        p.plot(1, tape7[:,0],selectHiTau * .2, plotCol='m',label=['High $\\tau$ selection'], linestyle='--', )
        p.plot(1, tape7[:,0],select3to5 * 0.4, plotCol='y',label=['3.5-4.8 $\\mu$m'], linestyle='--', )
        p.plot(1, tape7[:,0],select8to12 * 0.4, plotCol='c',label=['8.3-12 $\\mu$m'], linestyle='--', )
        p.plot(1, tape7[:,0],tape7[:,1],colselect[1],'Wavenumber cm$^{-1}$','Transmittance',
                    plotCol='r',maxNX=5,pltaxis=[0,4000,0,1])
        
        p.plot(2, tape7[:,0],tape7[:,2],colselect[2],'Wavenumber cm$^{-1}$','Radiance W/(m$^2$.sr.cm$^{-1}$)',
                    plotCol='r',label=['$L_\\mathrm{path}$'])
        p.plot(2, tape7[:,0],tape7[:,2]/(1.-tape7[:,1]),plotCol='b',label=['$L_\\mathrm{path}$ / $(1-\\tau)$'], 
                    maxNX=5)
        p.plot(2, tape7[:,0],selectHiTau * 0.05, plotCol='m',label=['High $\\tau$ selection'], linestyle='--', )
        p.plot(2, tape7[:,0],select3to5 * 0.1, plotCol='y',label=['3.5-4.8 $\\mu$m'], linestyle='--', )
        p.plot(2, tape7[:,0],select8to12 * 0.1, plotCol='c',label=['8.3-12 $\\mu$m'], linestyle='--', )
        p.plot(2, tape7[:,0],ryplanck.planck(tape7[:,0],tmpHiTau,'en')/np.pi,
                plotCol='g',label=['$L_\\mathrm{bb}$'+'({:.1f}) K (high $\\tau$)'.format(tmpHiTau)])
        p.plot(2, tape7[:,0],ryplanck.planck(tape7[:,0],tmpAll,'en')/np.pi,
                plotCol='k',label=['$L_\\mathrm{bb}'+'$({:.1f}) K (all)'.format(tmpAll)])
        p.plot(2, tape7[:,0],ryplanck.planck(tape7[:,0],tmpLoTau,'en')/np.pi,
                plotCol='c',label=['$L_\\mathrm{bb}$'+'({:.1f}) K (low  $\\tau$)'.format(tmpLoTau)], 
                    maxNX=5,pltaxis=[650,2500,0,0.15])        
        
        p.semilogY(3, tape7[:,0],tape7[:,1],colselect[1],'Wavenumber cm$^{-1}$','Transmittance',
                    plotCol='r',maxNX=5,pltaxis=[0,16000,1e-9,1e0])
        p.semilogY(4, tape7[:,0],tape7[:,2],colselect[2],'Wavenumber cm$^{-1}$','Radiance W/(m$^2$.sr.cm$^{-1}$)',
                    plotCol='r',label=['$L_\\mathrm{path}$'])
        p.semilogY(4, tape7[:,0],tape7[:,2]/(1.-tape7[:,1]),plotCol='b',label=['$L_\\mathrm{path}$ / $(1-\\tau)$'], 
                    maxNX=5,pltaxis=[0,16000,1e-30,1e0])
        p.semilogY(4, tape7[:,0],ryplanck.planck(tape7[:,0],tmpHiTau,'en')/np.pi,
                    plotCol='g',label=['$L_\\mathrm{bb}$'+'({:.1f}) K (high  $\\tau$)'.format(tmpHiTau)])
        p.semilogY(4, tape7[:,0],ryplanck.planck(tape7[:,0],tmpAll,'en')/np.pi,
                    plotCol='k',label=['$L_\\mathrm{bb}$'+'({:.1f}) K (all)'.format(tmpAll)])
        p.semilogY(4, tape7[:,0],ryplanck.planck(tape7[:,0],tmpLoTau,'en')/np.pi,
                    plotCol='c',label=['$L_\\mathrm{bb}$'+'({:.1f}) K (low  $\\tau$)'.format(tmpLoTau)], 
                    maxNX=5,pltaxis=[0,16000,1e-30,1e0])
       

In [ ]:
#to process and print atmospheric transmittance
dfAtmoTempk = dfAtmoTemp.sort_values(by=['Atmosphere','Altitude']).copy()

writer = pd.ExcelWriter('./StandardAtmoTemperatures.xlsx', engine='xlsxwriter')
dfAtmoTempk.to_excel(writer, sheet_name='Temperatures', startrow=0, startcol=0, header=True, index=False)

trimCols = ['TmpLoTau','TmpHiTau','3.5-4.8','8.3-12','TmpAve','TmpAll']
for col in trimCols:
    dfAtmoTempk[col] =  ((10*dfAtmoTempk[col]).astype(int))/10

atmolist = np.unique(dfAtmoTempk['Atmosphere'])
string = ''
for atmo in atmolist:
    dfAtmo = dfAtmoTempk[dfAtmoTempk.Atmosphere==atmo]
    dfAtmo = dfAtmo.set_index('Atmosphere')
    string += dfAtmo.to_html()
HTML(string)

In [ ]:
#to plot atmospheric temperature
r = ryplot.Plotter(1,1,1,'',figsize=(10,5))
atmoUnique = dfAtmoTempk['Atmosphere'].unique()
for atmos in atmoUnique:
    atmprofile = dfAtmoTempk[dfAtmoTempk.Atmosphere==atmos]
    r.plot(1, atmprofile['Altitude'], atmprofile['3.5-4.8'],xlabel='Altitude [m]',
           ylabel='Apparent temperature [K]',ptitle='Down-looking atmospheric temperature 3.5-4.8 $\mu$m',label=[atmos])
for alt in [1000, 5000, 30000]:
    malt = 0.3048 * alt
    r.plot(1,np.asarray([malt,malt]),np.asarray([240,320]),plotCol='k')
    
    cp = r.getSubPlot(1)
    cp.text(malt, 317, '{:.0f} kft'.format(alt/1000.), 
                  horizontalalignment='left', fontsize=8)

# Effective Transmittance in Spectral Bands

The effective transmittance in a number of spectral bands are calculated next.  The effective transmittance is calculated using 

\begin{equation}
\tau_\textrm{eff} = \frac{\int_{\lambda_1}^{\lambda_2} \tau_\lambda\,L_{\lambda\textrm{source}}\,d\lambda}{\int_{\lambda_1}^{\lambda_2} L_{\lambda\textrm{source}}\,d\lambda} = e^{-\gamma R}
\end{equation}
where the integration limits are defined by the spectral band, and the source radiance is calculated as a sum of reflected sunlight and emitted thermal radiation at 300 K, 

\begin{equation}
L_{\lambda\textrm{source}} = (1\,-\,\rho)\,L_\textrm{planck}(300K) + \xi\,\rho \,L_\textrm{planck}(6000K)
\end{equation}
where $\xi=2.1\times10^{-5}$ accounts for the sun's distance and area, $\rho$ is the surface reflection (default value 0.3) and $L_\textrm{planck}$ is Planck law radiance.  The exact value of reflectance is only important in the MWIR spectral band; in other bands the value divides out in the normalisation, and becomes unimportant.

The spectral bands are defined as follows:


In [ ]:
#to define the spectral bands
specranges = {}
with open('data/StandardSpectralRanges.txt','rt') as fin:
    lines = fin.readlines()
    for line in lines:
        linelst = line.rstrip().split()
        specranges[linelst[0]] = [float(linelst[1]),float(linelst[2])]
print(specranges)
import operator
specrangesSorted = [x[0] for x in sorted(specranges.items(), key=operator.itemgetter(1))]
print(specrangesSorted)

The function `analyseTau` calculates the effective transmittance and creates a Pandas dataframe with the results.

In [ ]:
#to calculate effective transmittance
def analyseTau(alts, dirs, atmospheres,specranges, reflect = 0.3, rangekm=5):
    """for specified atmospheres and altitudes calculate the effective transmittance
    for specified spectral bands, for sunlight radiance and 300K radiance 
    at the specified distance.
    Results are returned in a pandas dataframe
    """
        
    #prepare to create the data frame
    columns=['Atmosphere','Altitude','SpecRange','EffTauSlant','PathLenSlant','EffTauRange','PathLenRange','effGamma']
    aTau = pd.DataFrame(columns=columns)
    doPlot = True
    for i,directory in enumerate(dirs):
        for j, alt in enumerate(alts):
            #load data from file
            datadir = os.path.join('.',directory,'{}'.format(alt))
            filename = '{}-{}m.1km'.format(directory, alt)
            data = np.loadtxt(os.path.join(datadir,filename), skiprows=1)

            #calculate source radiance for sun and terrain
            LSun = ryplanck.planck(data[:,0],6000.,'el')
            L300 = ryplanck.planck(data[:,0],300.,'el')
            LEff = (1-reflect) * L300 + reflect * 2.1e-5 * LSun
            if doPlot:
                doPlot = False
                p = ryplot.Plotter(1,1,1,'Source Spectrum for Effective Transmittance Calculation',figsize=(12,5))
                p.logLog(1,data[:,0],LEff,'','Wavelength [$\mu$m]',
                         'Radiance [W/(m$^2$$\cdot$sr$\cdot$$\mu$m)]')#,pltaxis=[0.3,14,1,1000])
                    
            for k, spec in enumerate(specranges.keys()):
                #select a smaller spectral integration range according to spectral band definition
                select = np.zeros(data[:,0].shape)
                select = np.where(((data[:,0]>=specranges[spec][0]) & (data[:,0]<=specranges[spec][1])), 1.0, 0.0)
                # the following two lines check spectral selection
        #         seldiff = np.where(np.diff(select) != 0)
        #         print(specranges[spec][0], specranges[spec][1], data[seldiff,0])
                    
                #get slant path distance
                rangesl = ((alt/1000.) / (np.cos(45.0 * np.pi / 180.)) )

                taukm = np.exp( rangekm * np.log(data[:,1]))
                tausl = np.exp( rangesl * np.log(data[:,1]))
                effTaukm = np.trapz(select *LEff * taukm,data[:,0]) / np.trapz(select * LEff, data[:,0])
                effTaukm = int(1000.*effTaukm) / 1000.
                effTausl = np.trapz(select *LEff * tausl
                                    ,data[:,0]) / np.trapz(select * LEff, data[:,0])
                effGamma = -np.log(effTausl)/rangesl
                effTausl = int(1000.*effTausl) / 1000.
                aTau = aTau.append(pd.DataFrame([[directory,alt,spec,effTausl,rangesl,effTaukm,rangekm, effGamma]], columns=columns))
    return(aTau)    

Calculate the effective transmittance dataframe for the path length as shown below.  The source spectral radiance used in the calculation is also shown below.

In [ ]:
#to define atmospheric conditions
rangekm = 5 
reflect = 0.3
limAlts = alts
# limAlts = [305, 1524, 3048, 4572,  10668, 15240]
# limAlts = [4572]
aTau = analyseTau(limAlts, dirs, atmospheres, specranges, reflect, rangekm)
# print(aTau)


In [ ]:
#to sort and process the data
#first set up the dataframe to sort spectral range by the low wavelength, rather than alpha
# http://stackoverflow.com/questions/23482668/sorting-by-a-custom-list-in-pandas
specrangesSorted = sorted(specranges, key=lambda key: specranges[key][0])
aTau.SpecRange = aTau.SpecRange.astype("category")
aTau.SpecRange.cat.set_categories(specrangesSorted, inplace=True)


Next manipulate the data to extract and display the effective transmittance values in tabular form for the different altitudes.

The data in the following tables describe two different path types in a somewhat confusing manner. Please note the following:

1. Slant path from altitude to ground: The first table in the set provides the effective transmittance along the slant path, from the stated altitude to ground, with a slant angle of -45 deg.  The path length quoted here ranges from altitude to ground at a fixed slant angle.

2. Near horizontal path: The second table in the set provides the effective transmittance but scaled to a fixed path length (e.g. 5 km) from the stated altitude, but at some arbitrary slant angle that would give a path length of the stated fixed path length.  In other words the slant angle is adjusted to provide a path length of 5 km. This does not make sense for altitudes above 5 km, but the values are calculated anyway.

In [ ]:
#to build the result tables
string = ''
for alt in limAlts:
    filData = aTau[aTau['Altitude']==alt]
    filData = filData.drop('Altitude', axis=1)
    
    pathType = ['PathLenSlant', 'PathLenRange']
    effTType = ['EffTauSlant', 'EffTauRange']
    
    for i in [0,1]:
        redData = filData
        #get appropriate range and drop the original rangecolumns
        pathlen = 1000. * np.unique(redData[[(pathType[i])]].values)[0]
        redData = redData.drop('PathLenSlant', axis=1)
        redData = redData.drop('PathLenRange', axis=1)
        redData = redData.drop('effGamma', axis=1)
        if i == 0:
            redData = redData.drop('EffTauRange', axis=1)
        else:
            redData = redData.drop('EffTauSlant', axis=1)
            
        #build a long description of the atmosphere
        redData.Atmosphere = redData.Atmosphere.map(lambda x: ' '.join((x, atmospheres[x][0])))
        #first create the mulitindex row-wise
        redData = redData.set_index(['Atmosphere', 'SpecRange'])
        #then unstack the deepest row-indexes into columns and select the target sze and concept 
        redData =  redData.unstack(('SpecRange'))
        redData.reindex_axis(sorted(redData.columns), axis=1)

        if i == 0:
            string += r'<P>\newpage'
            string +=  '<P>\nAltitude {} m, slant path length {:.0f} m, (recalculated from 1 km data set)'.format(alt,pathlen)
        else:
            string +=  '<P>\nAltitude {} m, fixed path length {:.0f} m, (recalculated from 1 km data set)'.format(alt,pathlen)
        string += redData.to_html()

HTML(string)

# Effective transmittance and path radiance to space vs sensor zenith angle

Consider a set of paths to space from the sea level at varying zenith angles from vertically up, down to the horizon.  This section plots the effective transmittance and path radiance along these paths for the standard atmospheres.  The effective transmittance is calculated for source temperatures of 6000~K and 300~K.  The path radiance is calculated in radiant and photon rate units.

The data is pre-calculated with the function `domodtran-elevation.py` and the results stored in an Excel spreadsheet.

In [ ]:
#to load the data and print the column headings
dfEff = pd.read_excel('atmos-elevation-angles.xlsx')
print(dfEff.columns)
# print(dfEff.Zenith.unique())

The columns in the spreadsheet are as follows:

    u'Atmo' is the type of atmosphere
    u'Altitude' is the ground altitude
    u'Zenith' is the zenith angle
    u'SpecBand' is the spectral band 
    u'ToaWattTot' is the wideband top-of-atmosphere radiant irradiance
    u'ToaWatt' is the inband top-of-atmosphere radiant irradiance
    u'BoaWattTot' is the wideband bottom-of-atmosphere radiant irradiance
    u'BoaWatt' is the inband bottom-of-atmosphere radiant irradiance
    u'LpathWatt' is the inband path radiant radiance
    u'ToaQTot' is the wideband top-of-atmosphere photon rate irradiance
    u'ToaQ' is the inband top-of-atmosphere photon rate irradiance
    u'BoaQTot' is the wideband bottom-of-atmosphere photon rate irradiance
    u'BoaQ' is the inband bottom-of-atmosphere photon rate irradiance
    u'LpathQ' is the inband path photon rate radiance
    u'effTauSun' is the inband effective transmittance for a 6000 K source
    u'effTau300' is the inband effective transmittance for a 300 K source

where 'wideband' means over the full spectral range of 0.29 to 15.2 $\mu$m and 'inband' means the standard spectral band defined as follows:

In [ ]:
#ro print the spectral ranges
print(specranges)

The path radiance calculation does not include the sun's direct illumination, but it does include the halo scattering of sunlight.  The Modtran `tape6` file contains the following:

    LATITUDE AT H1ALT =                0.0000 DEG NORTH OF EQUATOR
    LONGITUDE AT H1ALT =               0.0000 DEG WEST OF GREENWICH

    SUBSOLAR LATITUDE =              -23.0700 DEG NORTH OF EQUATOR
    SUBSOLAR LONGITUDE =             359.1925 DEG WEST OF GREENWICH
    TIME (<0 UNDEF) =                       12.0000 GREENWICH TIME
    PATH AZIMUTH (FROM H1ALT TO H2ALT) =     0.0000 DEG EAST OF NORTH
    DAY OF THE YEAR =                       1

     EXTRATERRESTIAL SOURCE IS THE SUN

from which we see that the azimuth direction for the sensor elevation scans  in this section is  exactly North.  The sun is 23 degrees South of the equator, so the sun direction is outside (South) of the sensor elevation scan direction (North).   The sun is relatively close to the equator, so the halo effect is visible in the data.  The sky radiance close to the sun will be even higher than the value shown here (because the maximum value shown here is still 23 degrees from the sun).  

In [ ]:
#to plot the direct irradiance on ground level for the different spectral bands
atmodirs = dfEff.Atmo.unique()
altitudes = dfEff.Altitude.unique()
specBands = specrangesSorted
ipl = 0
for ic,specBand in enumerate(specBands):
    for il,alt in enumerate([altitudes[0]]):
        ipl += 1
        p = ryplot.Plotter(ipl,1,2,figsize=(12,4))
        for ip,lradu in zip([1,2],['W/m2','q/(s.m2)']):
            p.resetPlotCol()
            for ia,atmodir in enumerate(atmodirs):
                dft = dfEff[ (dfEff.Atmo==atmodir) & \
                             (dfEff.Altitude==alt) & \
                             (dfEff.SpecBand==specBand)  \
                ].copy()

                dft = dft.sort_values(by='Zenith')
                plotVal = dft.BoaWatt if lradu=='W/m2' else dft.BoaQ
                p.plot(ip, dft.Zenith, plotVal, '{} km {} sun zenith {} deg '.format(alt/1000.,specBand, -23),
                       'Sensor zenith angle deg','Apparent irradiance {}'.format(lradu), label=[atmodir] )

The next set of graphs show the path radiance for the different spectral bands and for different altitudes. At zero altitude only zenith angles above the horizon are displayed.  At high altitudes the path radiance for all lookup and lookdown angles are displayed.

1. Vis/NIR/SWIR:  The atmospheres with high aerosol content has high  path radiance closer to the vertical (nearer the sun), whereas the atmospheres with low aerosol content has lower path radiance near the vertical.  The ratio in path radiance near the vertical is more than double (closer to the sun it will be even higher).  

2. Vis/NIR/SWIR: At  zenith angles more than 40 degrees from the sun, there is relatively little difference in path radiance between the different atmospheres, except near the horizon where aerosol differences lead to differences in path radiance.

1. Vis/NIR/SWIR: Towards the horizon the path radiance of high aerosol atmospheres is _lower_ than low aerosol atmosphere path radiance. Visually, I suspect that with aerosol atmospheres will have a white/grey horizon, whereas a high aerosol atmosphere has a red/brown horizon.  This is probably because high aerosol atmospheres already lost much of the sunlight in the halo around the sun, with less photons available on the horizon.

1. MWIR/LWIR: the path radiance near the sun or at small zenith angles is relatively small compared to the  path radiance nearer the horizon.  The atmospheric temperature plays a major role: hot atmospheres have higher path radiance.  Winter atmospheres have MWIR path radiance generally below 0.5 W/(m\tsup{2}.sr), whereas the really hot atmospheres have MWIR path radiance exceeding 1 W/(m\tsup{2}.sr).

1. The MWIR path radiance is less strongly affected by absolute humidity, and more by temperature.

1. The LWIR path radiance is strongly affected by absolute humidity, but also by temperature (keep in mind that atmospheres at higher temperatures allows higher absolute humidity).



In [ ]:
#to plot the path radiance for the different spectral bands
atmodirs = dfEff.Atmo.unique()
altitudes = dfEff.Altitude.unique()
specBands = specrangesSorted
ipl = 0
for ic,specBand in enumerate(specBands):
    for il,alt in enumerate(altitudes):
        ipl += 1
        p = ryplot.Plotter(ipl,1,2,figsize=(12,4))
        for ip,lradu in zip([1,2],['W/(m2.sr)','q/(s.m2.sr)']):
            p.resetPlotCol()
            for ia,atmodir in enumerate(atmodirs):
                dft = dfEff[ (dfEff.Atmo==atmodir) & \
                             (dfEff.Altitude==alt) & \
                             (dfEff.SpecBand==specBand)  \
                ].copy()

                dft = dft.sort_values(by='Zenith')
                plotVal = dft.LpathWatt if lradu=='W/(m2.sr)' else dft.LpathQ
                p.plot(ip, dft.Zenith, plotVal, '{} km {} sun zenith {} deg '.format(alt/1000.,specBand, -23),
                       'Sensor zenith angle deg','Path radiance {}'.format(lradu), label=[atmodir] )

In [ ]:
#to plot the effective transmittance for the different atmospheres
atmodirs = dfEff.Atmo.unique()
specBands = specrangesSorted
altitudes = dfEff.Altitude.unique()
ipl = 0
for ia,atmodir in enumerate(atmodirs):
    for il,alt in enumerate(altitudes):
        ipl += 1
        p = ryplot.Plotter(ipl,1,2,figsize=(12,4))
        for specBand,plotCol in zip(specBands,['r','g','b','m','c',]):
            dft = dfEff[ (dfEff.Atmo==atmodir) & \
                         (dfEff.Altitude==alt) & \
                         (dfEff.SpecBand==specBand)  \
            ].copy()
            dft = dft.sort_values(by='Zenith')
            p.plot(1, dft.Zenith, dft.effTauSun, '{} km {} {} K target'.format(alt/1000., atmodir,6000),
                   'Sensor zenith angle deg','Transmittance', label=[specBand], plotCol=plotCol )
            p.plot(2, dft.Zenith, dft.effTau300, '{} km {} {} K target'.format(alt/1000.,atmodir,300),
                   'Sensor zenith angle deg','Transmittance', label=[specBand], plotCol=plotCol )

In [ ]:
#to plot the effective transmittance for the different spectral bands
atmodirs = dfEff.Atmo.unique()
specBands = specrangesSorted
altitudes = dfEff.Altitude.unique()
ipl = 0
for ic,specBand in enumerate(specBands):
    for il,alt in enumerate(altitudes):
        p = ryplot.Plotter(ipl,1,2,figsize=(12,4))
        ipl += 1
        for ip,targ in zip([1,2],[6000,300]):
            p.resetPlotCol()
            for ia,atmodir in enumerate(atmodirs):
                dft = dfEff[ (dfEff.Atmo==atmodir) & \
                             (dfEff.Altitude==alt) & \
                             (dfEff.SpecBand==specBand)  \
                ].copy()

                dft = dft.sort_values(by='Zenith')
                plotVal = dft.effTauSun if targ==6000 else dft.effTau300
                p.plot(ip, dft.Zenith, plotVal, '{} effective transmittance for {} K target'.format(specBand,targ),
                       'Sensor zenith angle deg','Transmittance', label=[atmodir] )

# $\alpha$, $\beta$  and $\gamma$ fits to standard atmospheres

Clients sometimes use simple equations to model atmospheric transmittance with range.  These models are not suitable for wideband spectral ranges.  This section investigates the goodness of fit of two such models.  Both these models apply for a uniform atmospheric path. Slant paths from any height above 2 km would not be accurately modelled. 

The $(\alpha,\beta)$ extinction model  is as follows:
\begin{equation}
\tau = \exp{\left[-\alpha R \left( \frac{10}{R}\right)^\beta \right]}
\end{equation}

The $(\gamma)$  or Beer (Bouguer) extinction model  is as follows:
\begin{equation}
\tau = \exp{\left[-\gamma R \right]}
\end{equation}

In the graph below the curves labelled 'Effective Transmittance' are the true transmittance variation against range.  The sets of curves labelled '$\alpha/\beta$ and $\gamma$ are the approximation curve fits.  

It is quite evident that the approximation curves are not very good approximations.  For the data shown here the $(\alpha,\beta)$ extinction model gives a reasonable fit over the distances considered (0.1 to 15 km), but beyond 15 km the error grows very quickly.  The $(\gamma)$ model only fits reasonably for a path length shorter than about 2 km.

In [ ]:
#to model the approximations
def tauAB(prange, alpha, beta):
    tau = np.exp(-alpha * prange * (10/prange) ** beta)
    return tau

def tauG(prange, gamma):
    tau = np.exp(-gamma * prange)
    return tau

In [ ]:
#to collect list of atmospheres
wlLo = 3.6 # um
wlHi = 5.0 # um
tempTarg = 300 # K
strScenario = r'$\Delta\lambda=${}-{} $\mu$m, T=300 K'.format(wlLo, wlHi, tempTarg )
fileslist = ryfiles.listFiles(root=r'.', 
                patterns='.*\.1km', recurse=1, return_folders=0, useRegex=True)
#filter only 305 m
filelist = []
for item in fileslist:
    if '305' in item:
        filelist.append(item)

In [ ]:
#to plot the approximations vs true results.
prange = np.linspace(0.1,15,100)
p = ryplot.Plotter(1,len(filelist)/2,2, figsize=(12,18))
alphabetagamma = []
for i,filename in enumerate(filelist):
    #do this for horixontal path (homogeneous atmosphere)
    atmotype = os.path.basename(filename).replace('-305m.1km','')
    atmosphere = '{}: {}'.format(atmotype,atmospheres[atmotype][0])
    atmo = np.loadtxt(filename, skiprows=1)
    # select only the limited spectral range
    atmo = atmo[np.all([ atmo[:,0]<=wlHi, atmo[:,0]>=wlLo], axis=0),:]
    gamma = -np.log(atmo[:,1]) / 1.0  
    tau = np.exp(-gamma.reshape(-1,1) * prange.reshape(1,-1) )
 
    radtau = ryplanck.planck(atmo[:,0], tempTarg, 'el')
    tauEff = np.trapz(radtau.reshape(-1,1) * np.exp(-gamma.reshape(-1,1) * prange.reshape(1,-1) ),
                      atmo[:,0],axis=0) / np.trapz(radtau ,atmo[:,0],axis=0)
    
    popt, pcov = curve_fit(tauAB, prange, tauEff, p0=(0.5, 0.5) )
    alpha = popt[0]
    beta = popt[1]
    alfabetaL = r'$\alpha$={:.4f}  $\beta$={:.4f}'.format(alpha,beta)
    
    popt, pcov = curve_fit(tauG, prange, tauEff, p0=(0.5) )
    gamma = popt[0]
    gammaL = r'$\gamma$={:.4f}'.format(gamma)
    alphabetagamma.append([atmosphere, alpha,beta,gamma ])
   
    title = '{} {}'.format(atmotype,strScenario)
    p.logLog(i+1,prange, tauEff,'','','', linestyle=['-'],plotCol='b',
             label=['Effective transmittance'])
    p.logLog(i+1,prange, tauAB(prange,alpha, beta),'','','',
             linestyle=['--'],plotCol='r',label=[alfabetaL])
    p.logLog(i+1,prange, tauG(prange,gamma),title,
             'Range [m]','Transmittance', linestyle=['--'],plotCol='g',
             label=[gammaL],pltaxis=[0,15,1e-4,1])
#     cp = p.getSubPlot(i+1)
#     cp.text(0.15, 2e-3, strScenario, 
#                   horizontalalignment='left', fontsize=10)
    

In [ ]:
#to prepare tables
string = ''
string += r'$(\alpha,\beta)$ parameters for a horizontal homogeneous path '
string += 'under conditions: {}. '.format(strScenario)
string += r'Data fitted over path range of 0.1 to 15 km. '
abdf = pd.DataFrame(alphabetagamma, columns=['Atmosphere',r'$\alpha$',r'$\beta$',r'$\gamma$'])
string += abdf.to_html()
HTML(string)

# Python and [module versions, and dates](http://nbviewer.ipython.org/github/jrjohansson/scientific-python-lectures/blob/master/Lecture-0-Scientific-Computing-with-Python.ipynb)

In [ ]:
%load_ext version_information
%version_information numpy, scipy, matplotlib, pyradi, pandas